In [1]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
review_id = 13676

def crawl_data(url, res_id):
    global review_id
    browser = webdriver.Chrome()
    browser.get(url)

    results = []

    # Chờ phần review hiển thị
    try:
        WebDriverWait(browser, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "review-item"))
        )
    except:
        print(f"⛔ Không tìm thấy review nào cho {url}")
        browser.quit()
        return results

    # Nhấn "Xem thêm bình luận" cho đến khi hết
    while True:
        try:
            load_more = WebDriverWait(browser, 5).until(
                EC.element_to_be_clickable(
                    (By.XPATH, '//a[@class="fd-btn-more"]/label[contains(text(),"Xem thêm bình luận")]'))
            )
            browser.execute_script("arguments[0].click();", load_more)
            WebDriverWait(browser, 5).until(  # Wait for new content to load
                lambda driver: len(driver.find_elements(By.CLASS_NAME, "review-item")) > len(results)
            )
        except Exception:
            break

    comments = browser.find_elements(By.CLASS_NAME, "review-item")

    # Xuất dữ liệu review
    for c in comments:
        try:
            username = c.find_element(By.CSS_SELECTOR, ".ru-username").text.strip()
            review_text = c.find_element(By.CSS_SELECTOR, ".rd-des span").text.strip()
            score_text = c.find_element(By.CSS_SELECTOR, ".review-points span").text.strip()

        except:
            username = "Người dùng ẩn danh"  # hoặc "Ẩn danh"
            review_text = "Đã bình luận ẩn danh"
            score_text = "0.0"

        review = {
            "review_id": review_id,
            "username": username,
            "review_text": review_text,
            "score_text": score_text,
            "res_id": res_id,
        }
        review_id = review_id + 1
        results.append(review)
    browser.close()

    return results


In [3]:
from urls import urls

results = []
for idx, url in enumerate(urls, start=499):
    data = crawl_data(url, idx)
    for comment in data:
        results.append(comment)

# Xuất JSON
import json
with open("restaurants_reviews_03.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

